In [9]:
%pwd

'e:\\EndToEnd\\01_MLOps\\research'

In [10]:
import os
os.chdir('../')

In [11]:
%pwd

'e:\\EndToEnd\\01_MLOps'

In [12]:
from dataclasses import dataclass
from pathlib import Path
from mlops_wine_quality.constants import *
from mlops_wine_quality.utils.common import read_yaml, create_dirs, get_size
import os
import urllib.request as request
import zipfile
from mlops_wine_quality import logger


In [18]:
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    DataIngestionConfig, when we getting the data, below mention return
    type should be (mean we have created custom return type )
    """
    root_dir: Path
    source_url: str
    local_source_file: Path
    unzip_file: Path

In [19]:
class ConfigManger:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH):
        
        self.config = read_yaml(config_filepath)

        create_dirs([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dirs([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_source_file= config.local_source_file,
            unzip_file= config.unzip_dir
        )
        return data_ingestion_config

# components

In [20]:
class DataIngestion:
    def __init__(self,
                  config = DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_source_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_source_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_source_file))}")

    def unzip_data(self):
        unzip_path = self.config.unzip_file
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_source_file, "r") as f:
            f.extractall(unzip_path)

# Pipeline

In [22]:
try:
    config = ConfigManger()
    dataIngestion_config = config.get_data_ingestion_config()
    dataIngestion = DataIngestion(config=dataIngestion_config)
    dataIngestion.download_data()
    dataIngestion.unzip_data()
except Exception as e:
    raise e

[2024-01-19 02:00:04,402: INFO: common: Yaml file loaded successfully]
[2024-01-19 02:00:04,404: INFO: common: Created directory at artifacts]
[2024-01-19 02:00:04,406: INFO: common: Created directory at artifacts/data_ingestion]
[2024-01-19 02:00:05,517: INFO: 3202707814: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F1D4:358C4:2BA4:62B8:65A99155
Accept-Ranges: bytes
Date: Thu, 18 Jan 2024 21:00:05 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990026-FJR
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1705611605.387704,VS0,VE257
Vary: Authorization,Accept-Encoding,Origin
Access-Cont

In [17]:
%pwd

'e:\\EndToEnd\\01_MLOps'